In [2]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from deep_sort_realtime.deepsort_tracker import DeepSort
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification
import torch
from IPython.display import clear_output
import torchvision.transforms as transforms

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
app = FaceAnalysis(name="buffalo_l")  # có detector, landmark, arcface
app.prepare(ctx_id=0)
tracker = DeepSort(max_age=10000000000000000000000000000, n_init=3, nn_budget=100)

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\Admin/.insightface\models\buffalo_l\w600k_r50.onnx recognition ['None', 3, 112, 112] 127.

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_st

In [4]:
video_path = r"E:\UET\HMI\Pj\Input\Trich doan 3 (ly truong - me mo).mp4"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
emotion_labels = ['Genuine Disgust', 'Posed Disgust', 'Genuine Happiness', 'Posed Happiness',
            'Genuine Fear', 'Posed Fear', 'Genuine Anger', 'Posed Anger',
            'Genuine Surprise', 'Posed Surprise', 'Genuine Sadness', 'Posed Sadness']
num_classes = len(emotion_labels)

In [6]:
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
model.classifier = torch.nn.Linear(in_features=768, out_features=num_classes, bias=True)
model.to(device)
model.load_state_dict(torch.load(r"E:\UET\HMI\Pj\models\vit_model.pth", map_location=device))
model.eval()

c:\Users\Admin\anaconda3\envs\dlib_env\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Admin\AppData\Local\Temp\ipykernel_22456\394440479.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.se

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [7]:
num_classes = 12
IMG_SIZE = 224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [8]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [9]:
cap = cv2.VideoCapture(video_path)
frame_id = 0
track_faces = {}
ax = plt.gca()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    faces = app.get(frame)
    detections = []
    for face in faces:
        x1, y1, x2, y2 = face.bbox.astype(int)
        conf = face.det_score
        if (x2 - x1) < 70 or (y2 - y1) < 70:
            continue  # Bỏ mặt nhỏ
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, face.embedding))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = track.to_ltrb()
        l, t, r, b = int(l), int(t), int(r), int(b)
        rect = plt.Rectangle((l, t), r - l, b - t, linewidth=2,
                         edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

        # Thêm track ID
        ax.text(l, t - 5, f'ID: {track_id}', color='yellow', fontsize=10, weight='bold',
                bbox=dict(facecolor='black', alpha=0.5, edgecolor='none'))
        face_crop = frame[t:b, l:r]
        if track_id not in track_faces:
            track_faces[track_id] = []
        track_faces[track_id].append((frame_id, face_crop))

    # Hiển thị bằng matplotlib trong Jupyter
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 6))
    plt.imshow(frame_rgb)
    plt.title(f"Frame: {frame_id} - Active Tracks: {len(tracks)}")
    plt.axis('off')
    plt.show()
    clear_output(wait=True)

cap.release()

# In thông tin
print(f"Tổng số người đã track: {len(track_faces)}")
for tid, seq in track_faces.items():
    print(f"Track ID {tid}: {len(seq)} frames")


KeyboardInterrupt: 